In [2]:
import numpy as np
import pandas as pd 

from typing import Any, Dict, List, Optional
import datetime as dt

import glob 
import os

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

## Data Source: https://www.hockey-reference.com/leagues/NHL_2022.html ##

path = '/Users/rschraeder/Desktop/Projects/StanleyCupPredictions/data/'

def file_access(path: str): 
    dataframes = []
    filenames = []
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            filenames.append(name)
            df = pd.read_csv(os.path.join(root, name))
            dataframes.append(df) 
    
    return dataframes

team_stats_df = file_access(path)[0] 
team_stats_updated = file_access(path)[1]
games_df = file_access(path)[2]

# Null value counts
# games_df.isna().sum() 
# team_stats_df.isna().sum()

In [3]:
team_stats_df.head()

,Rk,Team,AvAge,GP,W,L,OL,PTS,PTS%,GF,...,PIM/G,oPIM/G,S,S%,SA,SV%,SO,G,PPG,SHOOTOUTS
0,1,Florida Panthers*,27.8,82,58,18,6,122,0.744,337,...,10.1,10.8,3062,11.0,2515,0.904,5,579,117,7
1,2,Colorado Avalanche*,28.2,82,56,19,7,119,0.726,308,...,9.0,10.4,2874,10.7,2625,0.912,7,540,115,6
2,3,Carolina Hurricanes*,28.3,82,54,20,8,116,0.707,277,...,9.2,7.7,2798,9.9,2310,0.913,6,477,84,3
3,4,Toronto Maple Leafs*,28.4,82,54,21,7,115,0.701,312,...,8.6,8.5,2835,11.0,2511,0.900,7,564,105,4
4,5,Minnesota Wild*,29.4,82,53,22,7,113,0.689,305,...,10.8,10.8,2666,11.4,2577,0.903,3,554,116,9


In [4]:
games_df = games_df.rename(columns=({
    'Date': 'date', 'Visitor': 'away_team', 'Home': 'home_team', 'G': 'away_goals', 'G.1': 'home_goals', 'LOG': 'length_of_game_min'
}))
games_df = games_df[['date', 'away_team', 'away_goals', 'home_team', 'home_goals', 'length_of_game_min']]

# Transforming data
games_df['length_of_game_min'] = [i.replace(':', '') for i in games_df['length_of_game_min']]
games_df['length_of_game_min'] = [(int(i[0]) * 60) + int(i[1:]) for i in games_df['length_of_game_min']]

games_df.date = games_df.date.apply(pd.to_datetime)
games_df

,date,away_team,away_goals,home_team,home_goals,length_of_game_min
0,2021-10-12,Pittsburgh Penguins,6,Tampa Bay Lightning,2,153
1,2021-10-12,Seattle Kraken,3,Vegas Golden Knights,4,145
2,2021-10-13,Winnipeg Jets,1,Anaheim Ducks,4,149
3,2021-10-13,Chicago Blackhawks,2,Colorado Avalanche,4,152
4,2021-10-13,Vancouver Canucks,2,Edmonton Oilers,3,162
...,...,...,...,...,...,...
1307,2022-04-29,San Jose Sharks,0,Seattle Kraken,3,136
1308,2022-04-29,Vegas Golden Knights,7,St. Louis Blues,4,143
1309,2022-04-29,Boston Bruins,2,Toronto Maple Leafs,5,149
1310,2022-04-29,Calgary Flames,1,Winnipeg Jets,3,141


In [5]:
def encoding_game_outcome(dataset, away_output_colname: str, home_output_colname: str, away_goals: str, home_goals: str) -> List[int]: 
    
    dataset[f'{away_output_colname}'] = (dataset[f'{away_goals}'] - dataset[f'{home_goals}']).apply(lambda x: 1 if x > 0 else 0)
    dataset[f'{home_output_colname}'] = (dataset[f'{home_goals}'] - dataset[f'{away_goals}']).apply(lambda x: 1 if x > 0 else 0)
    
    return dataset
        
games_df = encoding_game_outcome(games_df, 'away_outcome', 'home_outcome', 'away_goals', 'home_goals')
games_df


# Output to CSV
games_df.to_csv(os.path.join(path, 'regular_season_clean.csv'), index=False)
games_df.head()

,date,away_team,away_goals,home_team,home_goals,length_of_game_min,away_outcome,home_outcome
0,2021-10-12,Pittsburgh Penguins,6,Tampa Bay Lightning,2,153,1,0
1,2021-10-12,Seattle Kraken,3,Vegas Golden Knights,4,145,0,1
2,2021-10-13,Winnipeg Jets,1,Anaheim Ducks,4,149,0,1
3,2021-10-13,Chicago Blackhawks,2,Colorado Avalanche,4,152,0,1
4,2021-10-13,Vancouver Canucks,2,Edmonton Oilers,3,162,0,1


In [6]:
# Team name cleaning
team_stats_df['Team'] = [str(i).replace('*', '') for i in team_stats_df['Team']]

# Creating Column for Total Goals 

team_stats_df['G'] = team_stats_df.GF + team_stats_df.GA 

# Creating Column for Total Power-Play Goals 

team_stats_df['PPG'] = team_stats_df.PP + team_stats_df.PPA

# Creating Column for Total Games in Shootouts

team_stats_df['SHOOTOUTS'] = team_stats_df.SOW + team_stats_df.SOL

def percents(df): 
    for column, row in df.iteritems(): 
        if '%' in column:
            for item in row: 
                if item < 1: 
                    row += row * 100
        
    return df

team_stats_df = percents(team_stats_df)

team_stats_df

# Output to CSV
team_stats_df.to_csv(os.path.join(path, 'team_stats_clean.csv'), index=False)

In [15]:
# Add the Stanley Cup Final Schedule
def add_fake_data(
    data, date, away_teams, home_teams, 
    away_goals, home_goals, away_result, home_result,
    game_length
): 
    fake_data = pd.DataFrame({
        'date': date, 
        'away_team': away_teams, 
        'home_team': home_teams, 
        'away_goals': away_goals,
        'home_goals': home_goals,
        'away_outcome': away_result,
        'home_outcome': home_result, 
        'length_of_game_min': game_length
    })
   
    result = pd.concat([data, fake_data], ignore_index=True)
    return result

dates = ['2022-06-15', '2022-06-18', '2022-06-20', '2022-06-22', '2022-06-24', '2022-06-26']
dates_list = [dt.datetime.strptime(date, '%Y-%m-%d') for date in dates]

away_teams = [
        'Tampa Bay Lightning', 
        'Tampa Bay Lightning', 
        'Colorado Avalanche', 
        'Colorado Avalanche', 
        'Tampa Bay Lightning', 
        'Colorado Avalanche'
]
home_teams = [
        'Colorado Avalanche', 
        'Colorado Avalanche', 
        'Tampa Bay Lightning', 
        'Tampa Bay Lightning', 
        'Colorado Avalanche', 
        'Tampa Bay Lightning'
    ]
stanley_cup_df = add_fake_data(
    data = games_df,
    date = dates_list,
    away_teams = away_teams,
    home_teams = home_teams,
    away_goals = [games_df.groupby('away_team')['away_goals'].mean().loc[i] for i in away_teams],
    home_goals = [games_df.groupby('home_team')['home_goals'].mean().loc[i] for i in home_teams],
    away_result = [games_df.groupby('away_team')['away_outcome'].mean().loc[i] for i in away_teams],
    home_result = [games_df.groupby('home_team')['home_outcome'].mean().loc[i] for i in home_teams],
    game_length = [games_df['length_of_game_min'].mean() for i in range(6)]
)
stanley_cup_df = encoding_game_outcome(stanley_cup_df, 'away_outcome', 'home_outcome', 'away_outcome', 'home_outcome')

# View preview
stanley_cup_df.tail(6)


,date,away_team,away_goals,home_team,home_goals,length_of_game_min,away_outcome,home_outcome
1312,2022-06-15,Tampa Bay Lightning,3.512195,Colorado Avalanche,4.195122,148.640244,0,1
1313,2022-06-18,Tampa Bay Lightning,3.512195,Colorado Avalanche,4.195122,148.640244,0,1
1314,2022-06-20,Colorado Avalanche,3.414634,Tampa Bay Lightning,3.487805,148.640244,0,1
1315,2022-06-22,Colorado Avalanche,3.414634,Tampa Bay Lightning,3.487805,148.640244,0,1
1316,2022-06-24,Tampa Bay Lightning,3.512195,Colorado Avalanche,4.195122,148.640244,0,1
1317,2022-06-26,Colorado Avalanche,3.414634,Tampa Bay Lightning,3.487805,148.640244,0,1


In [16]:
# Create full dataframe 
raw_cup_data = stanley_cup_df.merge(team_stats_df, how='left', left_on=('away_team'), right_on=('Team'))
raw_cup_data.drop(columns=({'Team'}), inplace=True)

clean_cup_data = stanley_cup_df.merge(team_stats_updated, how='left', left_on=('away_team'), right_on=('Team'))
clean_cup_data.drop(columns=({'Team'}), inplace=True)

# Output dataset with team names to a separate CSV. Updated CSV only.
clean_cup_data.to_csv(os.path.join(path, 'categorical_teams_set_cup.csv'), index=False)

# View preview
clean_cup_data.tail()

,date,away_team,away_goals,home_team,home_goals,length_of_game_min,away_outcome,home_outcome,Rk,AvAge,...,SA,SV%,SO,G,PPG,SHOOTOUTS,W%,GDIFF,SOW%,PPW%
1313,2022-06-18,Tampa Bay Lightning,3.512195,Colorado Avalanche,4.195122,148.640244,0,1,7,29.6,...,2441,91.607,3,513,111,7,62.195122,55.555556,28.571429,23.938224
1314,2022-06-20,Colorado Avalanche,3.414634,Tampa Bay Lightning,3.487805,148.640244,0,1,2,28.2,...,2625,92.112,7,540,115,6,68.292683,57.037037,66.666667,24.014337
1315,2022-06-22,Colorado Avalanche,3.414634,Tampa Bay Lightning,3.487805,148.640244,0,1,2,28.2,...,2625,92.112,7,540,115,6,68.292683,57.037037,66.666667,24.014337
1316,2022-06-24,Tampa Bay Lightning,3.512195,Colorado Avalanche,4.195122,148.640244,0,1,7,29.6,...,2441,91.607,3,513,111,7,62.195122,55.555556,28.571429,23.938224
1317,2022-06-26,Colorado Avalanche,3.414634,Tampa Bay Lightning,3.487805,148.640244,0,1,2,28.2,...,2625,92.112,7,540,115,6,68.292683,57.037037,66.666667,24.014337


# Classification of Numerical Variables

> Encoding categorical values
> Some features are still categorical and will be converted if deemed necessary, then I'll join the dataframes together to complete the dataset for my train and testing split

In [17]:
def encoding_full(
    df, home_team: str, away_team: str, date_col: str
): 
    
    # Encode non-numeric variables
    df.home_team = df.home_team.astype('category').cat.codes 
    df.away_team = df.away_team.astype('category').cat.codes
    
    # Encode date as a day of week to avoid time series implication.
    df.rename(columns={f'{date_col}': 'day_of_week'}, inplace=True)
    df['day_of_week'] = df['day_of_week'].dt.dayofweek
    
    return df

classification_df = encoding_full(
    clean_cup_data, 'home_team', 'away_team', 'date'
)

classification_df = classification_df[[
    'day_of_week', 'away_team', 'away_goals', 'home_team', 'home_goals', 'length_of_game_min',
    'away_outcome', 'home_outcome', 'W', 'L', 'GDIFF', 'SOW%', 'PTS', 'PTS%','PP', 'PP%'
]]

classification_df = classification_df.iloc[:,0:].astype(int)

# Output dataset with all encoded variables to training set. Will be used for training model. 
classification_df.to_csv(os.path.join(path, "new_encoded_variables.csv"), index=False)

# View preview
classification_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1318 entries, 0 to 1317
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   day_of_week         1318 non-null   int64
 1   away_team           1318 non-null   int64
 2   away_goals          1318 non-null   int64
 3   home_team           1318 non-null   int64
 4   home_goals          1318 non-null   int64
 5   length_of_game_min  1318 non-null   int64
 6   away_outcome        1318 non-null   int64
 7   home_outcome        1318 non-null   int64
 8   W                   1318 non-null   int64
 9   L                   1318 non-null   int64
 10  GDIFF               1318 non-null   int64
 11  SOW%                1318 non-null   int64
 12  PTS                 1318 non-null   int64
 13  PTS%                1318 non-null   int64
 14  PP                  1318 non-null   int64
 15  PP%                 1318 non-null   int64
dtypes: int64(16)
memory usage: 175.0 KB


In [9]:
# TODO: 

"""Find a way to extract data from the website and save it via this pipeline into the /data directory. There is a link on the website to save CSV files, so this can be accessed with selenium and exported to the directory OR be ideally fetched via API if available and stored as parquet in an S3 bucket. This is once I can get this on AWS. Clean this up and store the separate pieces of data in functions, maybe create dataclasses."""

'Find a way to extract data from the website and save it via this pipeline into the /data directory. There is a link on the website to save CSV files, so this can be accessed with selenium and exported to the directory OR be ideally fetched via API if available and stored as parquet in an S3 bucket. This is once I can get this on AWS. Clean this up and store the separate pieces of data in functions, maybe create dataclasses.'